# Advection

At first, we need to import the relevant Python libraries. Notably, ipywidgets provides a progress bar.

In [ ]:
from __future__ import print_function

import numpy as np
import scipy.sparse.linalg as sp_lin_alg
from scipy.sparse.linalg import LinearOperator
from ipywidgets import IntSlider

import os, sys, pyvista

Next, we set the global parameters. That is,
- theta: defining the theta time-stepping scheme,
- the polynomial degree of the local approximation spaces,
- the refinement level of the mesh,
- the number of time steps,
- the final time,
- the time-step size (deduced from the aforementioned information),
- and the amount of time steps between two output files that are written.

In [ ]:
theta       = 0.5
poly_degree = 0
refinement  = 5
debug_mode  = False

time_steps      = 10 ** 3
time_end        = 5
delta_time      = time_end / time_steps
output_interval = 20

Importing HyperHDG works as always: First, we check whether we can simply import it. If not, we add the path to the `import` folder and import it.

In [ ]:
try:
  import HyperHDG
except (ImportError, ModuleNotFoundError) as error:
  sys.path.append("../import")
  import HyperHDG

The HyperHDG.config contains all relevant information to define the problem,

In [ ]:
const                 = HyperHDG.config()
const.global_loop     = "Parabolic"
const.topology        = "File<2,3,std::vector,Point<3,double> >"
const.geometry        = "File<2,3,std::vector,Point<3,double> >"
const.node_descriptor = "File<2,3,std::vector,Point<3,double> >"
const.local_solver    = "AdvectionParab<2," + str(poly_degree) + "," + str(2*poly_degree) + ",LeVequeHG,double>"
const.cython_replacements = ["string", "string", "double", "vector[double]"]
const.include_files   = ["reproducibles_python/parameters/advection.hxx"]
const.debug_mode      = debug_mode

and is used to set it up.

In [ ]:
PyDP = HyperHDG.include(const)

HDG_wrapper = PyDP( "../domains/leVeque_hg.geo", lsol_constr= [0.,theta,delta_time] )
HDG_wrapper.refine( 2 ** refinement );

Having done so, we set the initial value and define Python's linear operator, which is neded for the linear solver later on.

In [ ]:
vectorSolution = HDG_wrapper.make_initial(HDG_wrapper.zero_vector())
  
system_size = HDG_wrapper.size_of_system()
A = LinearOperator( (system_size,system_size), matvec= HDG_wrapper.trace_to_flux )

Before we start the simulation, we write the initial state to a `.vtk` file and print its content within the notebook.

In [ ]:
HDG_wrapper.plot_option( "fileName" , "leVeque_hyg" + str(theta) + "-" + str(poly_degree) + "-" + str(refinement) )
HDG_wrapper.plot_option( "printFileNumber" , "true" )
# HDG_wrapper.plot_option( "scale" , "0.95" )
HDG_wrapper.plot_solution(vectorSolution, time_end)
grid = pyvista.read("output/leVeque_hyg" + str(theta) + "-" + str(poly_degree) + "-" + str(refinement) + ".0.vtu")
grid.plot(show_scalar_bar=True, show_axes=True);

The following code does the time stepping. Thus, it may run sime time, when started.

At first, it creates a progress bar/slider which illuminates the progress of the computation. Then, each time step is conducted by one repetition of the `for` loop and the solution is written to a `.vtk` file from time to time.

In [ ]:
progress_bar = IntSlider(min=0, max=time_steps, description='Times step:', readout=True)
display(progress_bar)
for time_step in range(time_steps):
  vectorRHS = np.multiply(HDG_wrapper.residual_flux(HDG_wrapper.zero_vector(), (time_step+1) * delta_time), -1.)
  
  [vectorSolution, num_iter] = sp_lin_alg.gmres(A,vectorRHS,tol=1e-13)
  if num_iter != 0:
    print("GMRES failed with a total number of ", num_iter, "iterations.")
    [vectorSolution, num_iter] = sp_lin_alg.bicgstab(A,vectorRHS,tol=1e-13)
    if num_iter != 0:
      print("BiCGStab also failed with a total number of ", num_iter, "iterations.")
      raise RuntimeError("All linear solvers did not converge!")
  
  HDG_wrapper.set_data(vectorSolution, (time_step+1) * delta_time)
  
  if (time_step+1) % output_interval == 0:
    HDG_wrapper.plot_solution(vectorSolution, time_end)
  progress_bar.value += 1

In the end, we calculate the L2 error and print it,

In [ ]:
error = HDG_wrapper.errors(vectorSolution, time_end)[0]
print( "Iteration: ", refinement, " Error: ", error )

before we write the final solution to a .vtk file and print it.

In [ ]:
f = open("output/advection_convergence_rotation_theta"+str(theta)+".txt", "a")
f.write("Polynomial degree = " + str(poly_degree) + ". Theta = " + str(theta) + ". Iteration = " + str(refinement) + ". Error = " + str(error) + ".\n")
f.close()
grid = pyvista.read("output/leVeque_hyg" + str(theta) + "-" + str(poly_degree) + "-" + str(refinement) + "." + str(int(time_steps/output_interval) + 1) + ".vtu")
grid.plot(show_scalar_bar=True, show_axes=True);